In [329]:

import numpy as np
s1 = 0.2
s2 = 0.3
s3 = 1 - s1 - s2
q = 0.2
r = 0.8
t = 0.6
q_begin = 0.1
r_begin = 0.9
t_begin = 0.5
q_eva = q_begin
r_eva = r_begin
t_eva = t_begin


In [330]:
#生成硬币序列
np.random.seed(1)

coin = np.random.rand(1000,100)
coin_sum = []
coin_class = np.random.rand(1000)
for i in range(coin.shape[0]):
    if coin_class[i] < s1: 
        #投掷硬币为s1
        coin[i][coin[i]>(1-q)] = 1
        coin[i][coin[i]<(1-q)] = 0
    elif coin_class[i] < s1 + s2:
        #投掷硬币为s2
        coin[i][coin[i]>(1-r)] = 1
        coin[i][coin[i]<(1-r)] = 0
    else:
        #投掷硬币为s2
        coin[i][coin[i]>(1-t)] = 1
        coin[i][coin[i]<(1-t)] = 0
    coin_sum.append(coin[i].sum())
coin_class[coin_class < s1] = 1 #a
coin_class[(coin_class > s1) & (coin_class < (s1 + s2))] = 2 #b
coin_class[(coin_class > (s1 + s2)) & (coin_class < 1)] = 3 #c
# print(coin_class)
# print(coin_sum)



In [331]:
# 根据硬币分布计算概率
def cal_probality(coin_seq, p):
    num_p = coin_seq.sum()
    num = coin_seq.size
    probality = (p**num_p) * (1-p)**(num-num_p)
    return probality


In [332]:
# EM算法
epoch = 10
for i in range(epoch):
    pro_list = []
    for j in range(coin.shape[0]):
        num_p = coin[j].sum()
        a_p = cal_probality(coin[j], q_eva)
        b_p = cal_probality(coin[j], r_eva)
        c_p = cal_probality(coin[j], t_eva)
        a_p_con =  a_p / (a_p+b_p+c_p)
        b_p_con =  b_p / (a_p+b_p+c_p)
        c_p_con =  c_p / (a_p+b_p+c_p)
        pro_list.append([a_p_con, b_p_con, c_p_con, num_p])
    a_p_num = 0
    a_n_num = 0
    b_p_num = 0
    b_n_num = 0
    c_p_num = 0
    c_n_num = 0
    for exp in pro_list:
        a_p_num += exp[0] * exp[3]
        a_n_num += exp[0] * (coin.shape[1]-exp[3])

        b_p_num += exp[1] * exp[3]
        b_n_num += exp[1] * (coin.shape[1]-exp[3])
        
        c_p_num += exp[2] * exp[3]
        c_n_num += exp[2] * (coin.shape[1]-exp[3])
    q_eva = a_p_num / (a_p_num + a_n_num)
    r_eva = b_p_num / (b_p_num + b_n_num)
    t_eva = c_p_num / (c_p_num + c_n_num)

    print("第%d轮迭代的概率为，a=%f,b=%f,c=%f"%(i+1, q_eva, r_eva, t_eva))
s1_eva = 0 
s2_eva = 0
s3_eva = 0
for exp in pro_list:
    s1_eva += exp[0] 
    s2_eva += exp[1]
    s3_eva += exp[2]
print("真实：q=%f,r=%f,t=%f"%(q, r, t))
print("初始：q=%f,r=%f,t=%f"%(q_begin, r_begin, t_begin))
print("预测：q=%f,r=%f,t=%f"%(q_eva, r_eva, t_eva))
print("真实:s1=%f,s2=%f,s3=%f"%(s1, s2, s3))
print("预测:s1=%f,s2=%f,s3=%f"%(s1_eva/coin_class.shape[0], s2_eva/coin_class.shape[0], s3_eva/coin_class.shape[0]))
    

第1轮迭代的概率为，a=0.196431,b=0.807482,c=0.595842
第2轮迭代的概率为，a=0.200104,b=0.801917,c=0.597484
第3轮迭代的概率为，a=0.200110,b=0.801108,c=0.597000
第4轮迭代的概率为，a=0.200109,b=0.800899,c=0.596866
第5轮迭代的概率为，a=0.200109,b=0.800843,c=0.596831
第6轮迭代的概率为，a=0.200109,b=0.800829,c=0.596822
第7轮迭代的概率为，a=0.200109,b=0.800825,c=0.596819
第8轮迭代的概率为，a=0.200109,b=0.800824,c=0.596819
第9轮迭代的概率为，a=0.200109,b=0.800823,c=0.596818
第10轮迭代的概率为，a=0.200109,b=0.800823,c=0.596818
真实：q=0.200000,r=0.800000,t=0.600000
初始：q=0.100000,r=0.900000,t=0.500000
预测：q=0.200109,r=0.800823,t=0.596818
真实:s1=0.200000,s2=0.300000,s3=0.500000
预测:s1=0.207012,s2=0.302225,s3=0.490763
